In [124]:
ATOM = 0
NOT = 1
OR = 2
AND = 3

class P(object):
    def __init__(self, *args, kind=ATOM):
        self.kind = kind
        self.args = args

    def __invert__(self):
        return P(self, kind=NOT)
    
    def __or__(self, other):
        return P(self, other, kind=OR)
    
    def __and__(self, other):
        return P(self, other, kind=AND)
    
    def to_str(self, atom_fun):
        if self.kind == ATOM:
            return atom_fun(self.args[0])
        if self.kind == NOT:
            return 'NOT {}'.format(self.args[0].to_str(atom_fun))
        if self.kind == OR:
            return '({} OR {})'.format(self.args[0].to_str(atom_fun), self.args[1].to_str(atom_fun))
        if self.kind == AND:
            return '({} AND {})'.format(self.args[0].to_str(atom_fun), self.args[1].to_str(atom_fun))  

def build_tag_query(p):
    atoms = {}
    atom_count = 1
    
    def walk(p):
        nonlocal atom_count
        nonlocal atoms
        if p.kind == ATOM:
            atoms[p.args[0]] = atom_count
            atom_count += 1
        else:
            for a in p.args:
                walk(a)
    
    walk(p)
                
    def atom_fun(a):
        return 'hash IN pred{}'.format(atoms[a])
    
    subqs = []
    params = []
    #need to check k[1] for safety here...
    for k, v in atoms.items():
        subqs.append('pred{} AS (SELECT hash FROM Tag WHERE key = ? AND value {} ?)'.format(v, k[1]))
        params.extend([k[0], k[2]])
    
    query = 'WITH {} SELECT * FROM Data WHERE {}'.format(', '.join(subqs), p.to_str(atom_fun))
    return (query, params)